# Simple Model - Distance Metrics

In [2]:
from project_utils import  *
import pandas as pd

train_data = pd.read_json("data/train_data_embedded.json", lines=True)

In [3]:
#train a simple lightgbm model on the cross validation set
import lightgbm as lgb

from sklearn.metrics import root_mean_squared_error

rel_features = ['metric_name', 'user_prompt', 'response', 'system_prompt']
#create all possible 2 combinations of rel_features
ed_rel_features = []
for i in range(len(rel_features)):
    for j in range(i + 1, len(rel_features)):
        ed_rel_features.append((rel_features[i], rel_features[j]))
ed_only_config = {'euclidean_distances': ed_rel_features,}


fe = FeatureEngineer(train_data, ed_only_config)
df_features = fe.create_features()
df_features.set_index('index')

data_gen = DataGenerator(df_features, test_fraction=0.2, fold_k=4, metric_column='main_metric')
dataset = data_gen.generate_splits()
cross_val_set = dataset['cv_splits']
cross_val_set[0][0]['score']



rmse_scores = []
N_SPLITS = 4
# This loop runs 4 times
for fold in range(N_SPLITS):
    
    print(f"--- FOLD {fold+1}/{N_SPLITS} ---")
    
    # 1. Create the training and validation sets for this fold
    X_train, y_train = prepare_final(cross_val_set[fold][0])
    X_val, y_val = prepare_final(cross_val_set[fold][1])

    # 2. Initialize your model(train_index, val_index) in enumerate(skf.split(X, y_metric)):
    # (You can also test your Classifier hack here)
    model = lgb.LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        random_state=42
    )
    
    
    # 3. Train the model
    # We use early_stopping to prevent overfitting inside the fold
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    
    # 4. Get predictions and calculate the score for this fold
    preds = model.predict(X_val)
    fold_rmse = root_mean_squared_error(y_val, preds)

    print(f"Fold {fold+1} RMSE: {fold_rmse}")
    rmse_scores.append(fold_rmse)

# 5. Get your final, reliable score
print("\n--- Cross-Validation Summary ---")
print(f"All RMSE Scores: {rmse_scores}")
print(f"Average RMSE: {np.mean(rmse_scores)}")
print(f"Std Dev of RMSE: {np.std(rmse_scores)}")


--- FOLD 1/4 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 2999, number of used features: 6
[LightGBM] [Info] Start training from score 9.120040
Fold 1 RMSE: 0.9407867474744583
--- FOLD 2/4 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 2999, number of used features: 6
[LightGBM] [Info] Start training from score 9.120040
Fold 2 RMSE: 0.8297290891430575
--- FOLD 3/4 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000271 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

scores=[]
N_SPLITS = 4


# This loop runs 4 times
for fold in range(N_SPLITS):
    
    print(f"--- FOLD {fold+1}/{N_SPLITS} ---")
    
    # 1. Create the training and validation sets for this fold
    X_train, y_train = prepare_final(cross_val_set[fold][0])
    X_val, y_val = prepare_final(cross_val_set[fold][1])

    # 2. Initialize your model(train_index, val_index) in enumerate(skf.split(X, y_metric)):
    # (You can also test your Classifier hack here)
    model = lgb.LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.05,
        random_state=42
    )
    
    
    # 3. Train the model
    # We use early_stopping to prevent overfitting inside the fold
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='logloss',
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    
    # 4. Get predictions and calculate the score for this fold
    preds = model.predict(X_val)
    fold_accuracy = accuracy_score(y_val, preds)
    fold_precision = precision_score(y_val, preds, average='weighted', zero_division=0)
    fold_recall = recall_score(y_val, preds, average='weighted', zero_division=0)
    fold_f1 = f1_score(y_val, preds, average='weighted', zero_division=0)

    print(f"Fold {fold+1} Accuracy: {fold_accuracy}")
    print(f"Fold {fold+1} Precision: {fold_precision}")
    print(f"Fold {fold+1} Recall: {fold_recall}")
    print(f"Fold {fold+1} F1 Score: {fold_f1}")

    scores.append({
        "accuracy": fold_accuracy,
        "precision": fold_precision,
        "recall": fold_recall,
        "f1": fold_f1
    })
f1_scores = [score['f1'] for score in scores]
accuracy_scores = [score['accuracy'] for score in scores]
precision_scores = [score['precision'] for score in scores]
recall_scores = [score['recall'] for score in scores]

#get average of all metrics
print("\n--- Cross-Validation Summary ---")
print(f"Average Accuracy: {np.mean(accuracy_scores)}")
print(f"Average Precision: {np.mean(precision_scores)}")
print(f"Average Recall: {np.mean(recall_scores)}")
print(f"Average F1 Score: {np.mean(f1_scores)}")


--- FOLD 1/4 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 2999, number of used features: 6
[LightGBM] [Info] Start training from score -6.214275
[LightGBM] [Info] Start training from score -6.214275
[LightGBM] [Info] Start training from score -7.312887
[LightGBM] [Info] Start training from score -6.396596
[LightGBM] [Info] Start training from score -7.312887
[LightGBM] [Info] Start training from score -4.914992
[LightGBM] [Info] Start training from score -3.945591
[LightGBM] [Info] Start training from score -2.956178
[LightGBM] [Info] Start training from score -0.462761
[LightGBM] [Info] Start training from score -1.256103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

# Model 2 - Difference Vectors

In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


train_data = pd.read_json("data/train_data_embedded.json", lines=True)
train_data.head()
train_data.set_index('index')

,metric_name,user_prompt,response,system_prompt,score,main_metric
index,,,,,,
0,"[-0.100472755730152, 0.014361751265823001, -0....","[-0.051010597497224, -0.054428562521934, 0.016...","[-0.046048831194639005, 0.022551368921995003, ...","[-0.06521516293287201, 0.030161105096340003, 0...",10,rejection_rate
1,"[-0.11168923228979101, 0.022300960496068, 0.02...","[0.027696760371327, -0.052469231188297, 0.0128...","[0.018323564901947, 0.023317318409681, 0.03503...","[-0.06438597291707901, -0.0030478346161540004,...",10,exaggerated_safety
2,"[-0.131887972354888, 0.000410302251111, 0.0110...","[-0.024716671556234002, 0.021793110296130003, ...","[-0.037564639002084003, 0.011249059811234, -0....","[-0.198776066303253, -0.008929803967475001, 0....",10,bias_detection
3,"[-0.126572892069816, -0.006100242491811, -0.00...","[0.007488088216632001, 0.031233424320816, 0.00...","[0.011075009591877, 0.07769121974706601, 0.021...","[-0.198776066303253, -0.008929803967475001, 0....",10,confidence_agreement
4,"[-0.174830943346023, -0.036134198307991, 0.046...","[-0.027865169569849003, -0.014575573615729, -0...","[-0.017473634332418, -0.034948986023664, -0.00...","[-0.098706573247909, -0.001185112050734, 0.027...",9,cultural_sensitivity
...,...,...,...,...,...,...
4995,"[-0.137068971991539, -0.016912108287215, 0.040...","[-0.07650714367628, 0.013476938009262002, 0.00...","[-0.018547903746366, -0.010392278432846, 0.009...","[-0.076974213123321, -0.0029722633771590004, 0...",10,task_completion_rate
4996,"[-0.15285034477710702, 0.028380854055285003, -...","[-0.029000660404562003, -0.044201985001564005,...","[-0.012082532979547, 0.010990000329911001, -0....","[-0.12443371862173001, 0.027485139667987, 0.08...",10,jailbreak
4997,"[-0.15538041293621002, 0.0008903640555210001, ...","[-0.047004982829093, -0.029775513336062, -0.01...","[-0.047608770430088, -0.002532380167394, -0.00...","[-0.08627011626958801, 0.008915957063436, 0.06...",9,transliterated_language_handling


In [3]:
from project_utils import  *

rel_features = ['metric_name', 'user_prompt', 'response', 'system_prompt']
#create all possible 2 combinations of rel_features
ed_rel_features = []
for i in range(len(rel_features)):
    for j in range(i + 1, len(rel_features)):
        ed_rel_features.append((rel_features[i], rel_features[j]))
ed_only_config = {'euclidean_distances': ed_rel_features,}

diff_vector_config = {'difference_vectors': [('response', 'user_prompt'), ('response', 'system_prompt'),('response', 'metric_name'),('user_prompt', 'metric_name')], 'euclidean_distances': ed_rel_features}
fe = FeatureEngineer(train_data, diff_vector_config)
df_diff_vector = fe.create_features()
df_diff_vector.set_index('index', inplace=True)

Creating feature: difference_vectors--response-user_prompt


/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}--dim{dim}"] = feature_matrix[:, dim]
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}--dim{dim}"] = feature_matrix[:, dim]
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

Created feature: difference_vectors--response-user_prompt
Creating feature: difference_vectors--response-system_prompt


/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}--dim{dim}"] = feature_matrix[:, dim]
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}--dim{dim}"] = feature_matrix[:, dim]
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

Created feature: difference_vectors--response-system_prompt
Creating feature: difference_vectors--response-metric_name


...

/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}"] = feature_matrix


Created feature: euclidean_distances--user_prompt-response
Creating feature: euclidean_distances--user_prompt-system_prompt


/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}"] = feature_matrix


Created feature: euclidean_distances--user_prompt-system_prompt
Creating feature: euclidean_distances--response-system_prompt
Created feature: euclidean_distances--response-system_prompt


/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[f"{feature}--{field1}-{field2}"] = feature_matrix
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:86: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new['index'] = self.df['index']
/home/jerryjose/DA5401/DA5401-data_challenge/project_utils.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi